In [1]:
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('/content/final step data.csv')
df=df.drop(columns=['mmr','seller'])
print(df.shape)
m_df=df

(548535, 7)


In [2]:
#df['condition']=df['condition']/10
print(df.shape)

print(df.info())
df.sample(2)

(548535, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548535 entries, 0 to 548534
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          548535 non-null  int64  
 1   make          548535 non-null  object 
 2   model         548535 non-null  object 
 3   body          548535 non-null  object 
 4   condition     548535 non-null  float64
 5   odometer      548535 non-null  float64
 6   sellingprice  548535 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 29.3+ MB
None


,year,make,model,body,condition,odometer,sellingprice
388414,2012,nissan,altima,sedan,36.0,40215.0,14900.0
139637,2013,chevrolet,silverado 1500,crew cab,45.0,38455.0,31600.0


In [3]:
df.describe()
df=df.dropna()

In [4]:
target_transformer = PowerTransformer(method='yeo-johnson')
from sklearn.preprocessing import FunctionTransformer
from sklearn.svm import SVR


X = df.drop('sellingprice', axis=1)
y = df['sellingprice']
y_transformed =FunctionTransformer(np.log1p).fit_transform(y) #target_transformer.fit_transform(y.values.reshape(-1, 1)).ravel()  # flatten

numerical_features = ['condition', 'odometer']     # Numerical columns
categorical_features = ['make', 'model', 'body']  # Categorical columns
year_col = ['year']  # Year is numerical but treated separately


#inp_trns=PowerTransformer(method='yeo-johnson')

In [5]:

# Step 4: Define Preprocessor
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numerical_transformer = Pipeline(steps=[

    ('log1p', FunctionTransformer(np.log1p)),

   # ('power',PowerTransformer(method='yeo-johnson')) ,
    ('scaler', StandardScaler())
])


year_transformer = Pipeline(steps=[
    ('log1p', FunctionTransformer(np.log1p)),
    #('power', PowerTransformer(method='yeo-johnson')),
    ('scaler', StandardScaler())
])


preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_features),
    ('cat', categorical_transformer, categorical_features),
    ('year', year_transformer, year_col)
])





In [7]:

from sklearn.pipeline import Pipeline
from lightgbm import LGBMRegressor


In [10]:
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor
# Step 5: Define Pipeline
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('pca',PCA(n_components=70) ),
('regressor', LGBMRegressor(n_estimators=200, learning_rate=0.05, max_depth=7))


    # ('regressor',XGBRegressor(colsample_bytree=0.7272013899887455,
    #     gamma=0.22010384905535352,
    #     learning_rate=0.07838054876258249,
    #     max_depth=8,
    #     n_estimators=220,
    #     reg_lambda=1.6360295318449862,
    #     subsample=0.9582191749769029
    #                           ))
])

In [ ]:
# scores = cross_val_score(pipeline, X, y_transformed, cv=5, scoring='neg_mean_squared_error')
# rmse_scores = np.sqrt(-scores)
# print("Cross-Validated RMSE:", rmse_scores)
# print("Average RMSE:", np.mean(rmse_scores))


In [9]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from scipy.stats import randint, uniform
# Step 9: Fit full pipeline
pipeline.fit(X, y_transformed)
# param_dist = {
#     'regressor__n_estimators': randint(100, 400),
#     'regressor__max_depth': randint(3, 10),
#     'regressor__learning_rate': uniform(0.01, 0.3),
#     'regressor__subsample': uniform(0.7, 0.3),
#     'regressor__colsample_bytree': uniform(0.6, 0.4),
#     'regressor__gamma': uniform(0, 2),
#     'regressor__reg_lambda': uniform(0, 2)# L2 regularization
# }
# random_search = RandomizedSearchCV(
#     pipeline,
#     param_distributions=param_dist,
#     n_iter=30,              # कितने combinations test करने हैं
#     scoring='r2',
#     cv=5,
#     verbose=2,
#     n_jobs=-1,
#     random_state=42
# )
# random_search.fit(X, y_transformed)
# #Step 6: Best results
# print("Best R² Score:", random_search.best_score_)
# print("Best Parameters:", random_search.best_params_)


# Optional: Evaluate
y_pred = cross_val_predict(pipeline, X, y_transformed, cv=5)
print("\nEvaluation Metrics:")
print("RMSE:", np.sqrt(mean_squared_error(y_transformed, y_pred)))
print("MAE:", mean_absolute_error(y_transformed, y_pred))
print("R2 Score:", r2_score(y_transformed, y_pred))

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.442304 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17850
[LightGBM] [Info] Number of data points in the train set: 548535, number of used features: 70
[LightGBM] [Info] Start training from score 9.220056


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.351176 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17850
[LightGBM] [Info] Number of data points in the train set: 438828, number of used features: 70
[LightGBM] [Info] Start training from score 9.260753


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.366509 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17850
[LightGBM] [Info] Number of data points in the train set: 438828, number of used features: 70
[LightGBM] [Info] Start training from score 9.201986


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.582899 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17850
[LightGBM] [Info] Number of data points in the train set: 438828, number of used features: 70
[LightGBM] [Info] Start training from score 9.201911


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



Evaluation Metrics:
RMSE: 0.2714247291632967
MAE: 0.17687722874708373
R2 Score: 0.9108740777057326


In [11]:
# Step 10: Save pipeline & target transformer
# best_pipeline = random_search.best_estimator_
import pickle
with open("lgbmrt3hypara_model_pipeline.pkl", "wb") as file:
    pickle.dump(pipeline, file)
# with open("inp_transformer.pkl", "wb") as f:
#     pickle.dump(inp_trns, f)

with open("targe-parat_transformer.pkl", "wb") as f:
    pickle.dump(target_transformer, f)

# Optional: Evaluate